In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from pathlib import Path

2023-10-20 14:52:43.529878: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("train*.tfrecords"))

BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
    'hospital_expire_flag': tf.io.FixedLenFeature([], tf.float32),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['hospital_expire_flag']
    return (ecg_data, example['age'], example['gender']), label

@tf.function
def drop_na_labels(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(y))

@tf.function
def drop_na_age(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(x[1]))

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.filter(drop_na_labels)
    dataset = dataset.filter(drop_na_age)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

In [5]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

In [13]:
model = keras.models.load_model("data/models/resnet-age.keras")

In [7]:
for i in range(len(model.layers)):
    model.layers[i].trainable = False

In [8]:
model.layers[-2]

In [14]:
x = model.layers[-2].output
age_input = tf.keras.layers.Input(shape=(1,), name="age_input")
gender_input = tf.keras.layers.Input(shape=(1,), name="gender_input")
x = keras.layers.Add(name="merge")([x, age_input, gender_input])
x = tf.keras.layers.Dense(128, name="tl_dense_3")(x)
x = tf.keras.layers.Dense(1, activation='sigmoid', name = "tl_dense_4")(x)

new_model = keras.Model([model.input, age_input, gender_input],outputs=x)

new_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

In [15]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(make_checkpoint_dir("data/models", "resnet-tl-mort"))
]

new_model.fit(train_dataset, epochs=3, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/3
    453/Unknown - 112s 211ms/step - loss: 172.0059 - accuracy: 0.7829 - auc_1: 0.4978

2023-10-20 15:01:59.281648: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 345770336204284317
2023-10-20 15:01:59.285860: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 17960276964877780189
2023-10-20 15:01:59.285869: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 7082522387522850163
2023-10-20 15:01:59.285903: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 11900836301288335132
2023-10-20 15:03:36.794085: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 11091614813116197373
2023-10-20 15:03:36.799219: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 2711414014925879322
2023-10-20 15:03:36.799226: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv i

INFO:tensorflow:Assets written to: data/models/resnet-tl-mort-2023-10-20_15-00-06/assets


INFO:tensorflow:Assets written to: data/models/resnet-tl-mort-2023-10-20_15-00-06/assets


453/453 [==============================] - 214s 435ms/step - loss: 172.0059 - accuracy: 0.7829 - auc_1: 0.4978 - val_loss: 5.7356 - val_accuracy: 0.8817 - val_auc_1: 0.5000 - lr: 0.0010
Epoch 2/3
309/453 [===================>..........] - ETA: 31s - loss: 1.9363 - accuracy: 0.7929 - auc_1: 0.4938

KeyboardInterrupt: 